In [ ]:
import pandas as pd
import numpy as np
import cv2
from tensorflow.keras.models import load_model, Model
from tensorflow.keras.layers import Dense, Dropout

# Load the saved model
base_model = load_model('units/icon_counter.h5')

# Freeze the layers of the base model
for layer in base_model.layers:
    layer.trainable = False

# Get the output of the last layer before the output layers
# We need to find the last layer that is not an output layer
# Let's assume it's the layer before the output layers
last_layer = base_model.layers[-13]  # Adjust this index based on your model's architecture
last_layer_output = last_layer.output

# Add new layers for the new task
x = Dense(64, activation='relu')(last_layer_output)
x = Dropout(0.2)(x)
x = Dense(32, activation='relu')(x)
predictions = Dense(1, activation='linear', name='power_prediction')(x)

# Create the new model
new_model = Model(inputs=base_model.input, outputs=predictions)

# Compile the new model
new_model.compile(optimizer='adam', loss='mse', metrics=['mae'])

# Load the data from the CSV file
data = pd.read_csv('path_to_your_csv_file.csv')  # Replace with the actual path to your CSV file

# Assuming the CSV file has columns 'image_path' and 'target'
image_paths = data['image_path'].values
targets = data['target'].values

# Function to load and preprocess images
def load_and_preprocess_image(path):
    img = cv2.imread(path, cv2.IMREAD_GRAYSCALE)
    img = cv2.resize(img, (1920, 1080))  # Resize to match the model's input shape
    img = img.astype(np.float32) / 255.0  # Normalize
    img = np.expand_dims(img, axis=-1)  # Add channel dimension
    return img

# Load and preprocess images
images = np.array([load_and_preprocess_image(path) for path in image_paths])

# Train the new model
new_model.fit(images, targets, batch_size=32, epochs=10, verbose=1)

# Save the new model
new_model.save('units/power_predictor.h5')
